# Deep Research Agent with OpenHands SDK

This notebook demonstrates a **3-phase deep research workflow**:

| Phase | Agent | Output |
|-------|-------|--------|
| **1. Planning** | GPT-4o creates plan → GPT-5.1 critiques → iterate until approved | `research_plan.md` |
| **2. Research** | GPT-4o executes searches, gathers raw findings | Updated `research_plan.md` |
| **3. Synthesis** | GPT-5.1 synthesizes all findings into comprehensive report | `research_report.md` |

**Features:**
- ✅ Multi-agent collaboration (GPT-4o + GPT-5.1)
- ✅ Iterative plan refinement with scoring
- ✅ **Checkpointing** - Resume from crash/interrupt
- ✅ **Observability** - Laminar tracing (set `LMNR_PROJECT_API_KEY`)
- ✅ Professional report synthesis with citations

## Step 1: Setup (same as notebook 00)

In [ ]:
#!pip install openhands-sdk python-dotenv tavily-python -q

In [ ]:
import os
import json
import time
from dotenv import load_dotenv
load_dotenv()

from typing import List
from pydantic import Field
from tavily import TavilyClient
from openhands.sdk import LLM, Agent, Conversation, Tool, Action, Observation, ToolDefinition, TextContent
from openhands.sdk.tool import register_tool, ToolExecutor
from openhands.tools.terminal import TerminalTool
from openhands.tools.file_editor import FileEditorTool
from openhands.tools.task_tracker import TaskTrackerTool

print(f"Model: {os.getenv('LLM_MODEL', 'openai/gpt-4o')}")

# Observability: auto-enabled if LMNR_PROJECT_API_KEY is set
if os.getenv("LMNR_PROJECT_API_KEY"):
    print("✓ Observability: Laminar tracing enabled (view at laminar.sh)")
else:
    print("ℹ Observability: Set LMNR_PROJECT_API_KEY for tracing")

## Step 2: Create LLM and Tavily Tool

In [ ]:
# Create LLM
llm = LLM(
    model="openai/gpt-4o",
    api_key=os.getenv("LLM_API_KEY"),
    base_url=os.getenv("LLM_BASE_URL", None),
)

# Create Tavily client and tool (same as notebook 00)
tavily = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))

class SearchAction(Action):
    query: str = Field(description="Search query")

class SearchObservation(Observation):
    results: str = Field(description="Results")
    @property
    def to_llm_content(self): return [TextContent(text=self.results)]

class SearchExecutor(ToolExecutor):
    def __call__(self, action: SearchAction, conversation=None) -> SearchObservation:
        try:
            r = tavily.search(query=action.query, max_results=5)
            text = "\n\n".join([
                f"**{x['title']}**\n{x['content'][:300]}\nSource: {x['url']}"
                for x in r['results']
            ])
            return SearchObservation(results=text or "No results found")
        except Exception as e:
            # Error recovery: return error message instead of crashing
            return SearchObservation(results=f"Search failed: {str(e)}. Try a different query.")

class SearchTool(ToolDefinition[SearchAction, SearchObservation]):
    @classmethod
    def create(cls, conv_state) -> List["SearchTool"]:
        return [cls(description="Web search via Tavily (returns results with citations)",
                    action_type=SearchAction, observation_type=SearchObservation, 
                    executor=SearchExecutor())]

register_tool("TavilySearch", SearchTool.create)
print("✓ LLM and Tavily tool ready")

## Step 3: Create Agent with File Editor

The key difference from notebook 00: we include `FileEditorTool` so the agent can **create and update the research plan file**.

In [ ]:
agent = Agent(
    llm=llm,
    tools=[
        Tool(name=FileEditorTool.name),     # Read/write research_plan.md
        Tool(name=TaskTrackerTool.name),    # Track task completion
        Tool(name="TavilySearch"),          # Web search with citations
    ],
)
print("✓ Agent ready with tools:", [t.name for t in agent.tools])

## Step 4: The Deep Research Prompt

This is where the magic happens. We give the agent explicit instructions to:
1. **Create a plan file** before starting research
2. **Update the plan** as each task completes
3. **Iterate and refine** if initial searches don't yield enough info

In [ ]:
DEEP_RESEARCH_PROMPT = """
You are a deep research agent. Your task is to thoroughly research a topic using a structured, iterative approach.

## Your Process:

### Phase 1: Planning
1. Create a file called `research_plan.md` with:
   - The main research question
   - 3-5 specific sub-questions to investigate
   - A checklist of tasks (use markdown checkboxes: `- [ ]` and `- [x]`)

### Phase 2: Research
For each sub-question:
1. Read `research_plan.md` to check what's done and what's next
2. Use TavilySearch to find relevant information
3. Take notes on key findings (you can add them to the plan file)
4. Mark the task as complete in `research_plan.md` (change `- [ ]` to `- [x]`)

### Phase 3: Refinement
After initial research:
1. Review all findings
2. Identify any gaps or areas needing more depth
3. Add new tasks to `research_plan.md` if needed
4. Continue researching until satisfied

### Phase 4: Synthesis
1. Read the final `research_plan.md` 
2. Create a comprehensive summary with:
   - Key findings organized by theme
   - Important sources (URLs)
   - Conclusions and implications

## Rules:
- ALWAYS create and update `research_plan.md` - this is your memory
- ALWAYS include source URLs in your findings (citations)
- If a search fails, try rephrasing the query
- Be thorough but efficient - don't repeat searches

Now research: {topic}
"""

print("✓ Deep research prompt ready")

## Step 5: Create Critique Agent (GPT-5.1)

Following the [OpenHands iterative refinement pattern](https://docs.openhands.dev/sdk/guides/iterative-refinement), we use a second model to critique and improve the research plan before execution.

In [ ]:
# Create a separate LLM for critique (GPT-5.1 for stronger reasoning)
critique_llm = LLM(
    model="openai/gpt-5.1",
    api_key=os.getenv("LLM_API_KEY"),
    base_url=os.getenv("LLM_BASE_URL", None),
)

# Critique agent only needs file access (no search needed)
critique_agent = Agent(
    llm=critique_llm,
    tools=[
        Tool(name=FileEditorTool.name),  # Read/write plan and critique files
    ],
)

print(f"✓ Critique agent ready (model: {critique_llm.model})")

## Step 6: Run the Research Pipeline

The workflow with **checkpointing for resume capability**:

1. **Phase 1 - Planning**: Create plan → Critique → Improve → Loop
2. **Phase 2 - Research**: Execute Tavily searches for each sub-question
3. **Phase 3 - Synthesis**: GPT-5.1 writes comprehensive report

💾 **Checkpointing**: Progress is saved to `checkpoint.json` after each phase. If the notebook crashes or is interrupted, re-run this cell to resume from where you left off.

In [ ]:
import re

# Configuration
QUALITY_THRESHOLD = 80.0
MAX_ITERATIONS = 2
cwd = os.getcwd()
CHECKPOINT_FILE = "checkpoint.json"

# Choose your research topic
topic = "What are the latest breakthroughs in AI agents and autonomous systems in 2024-2025?"

# ===========================================
# Checkpoint System for Resume Capability
# ===========================================

def save_checkpoint(phase: str, iteration: int = 0):
    """Save current progress to enable resume after crash/interrupt."""
    checkpoint = {
        "phase": phase,
        "iteration": iteration,
        "topic": topic,
        "timestamp": time.strftime("%Y-%m-%d %H:%M:%S")
    }
    with open(CHECKPOINT_FILE, "w") as f:
        json.dump(checkpoint, f, indent=2)
    print(f"  💾 Checkpoint saved: {phase}")

def load_checkpoint():
    """Load checkpoint if exists."""
    try:
        with open(CHECKPOINT_FILE, "r") as f:
            return json.load(f)
    except FileNotFoundError:
        return None

def clear_checkpoint():
    """Remove checkpoint after successful completion."""
    if os.path.exists(CHECKPOINT_FILE):
        os.remove(CHECKPOINT_FILE)

# Check for existing checkpoint
checkpoint = load_checkpoint()
if checkpoint:
    print(f"🔄 Found checkpoint from {checkpoint['timestamp']}")
    print(f"   Phase: {checkpoint['phase']}, Iteration: {checkpoint.get('iteration', 0)}")
    print(f"   Topic: {checkpoint['topic'][:50]}...")
    resume_from = checkpoint['phase']
else:
    resume_from = None
    print("📝 Starting fresh research (no checkpoint found)")

# Prompts
PLAN_CREATION_PROMPT = """
Create a research plan in `research_plan.md` for the topic: {topic}

The plan should include:
1. Main research question
2. 4-5 specific sub-questions to investigate
3. A checklist of search tasks (use `- [ ]` format)

Just create the plan file - don't execute any searches yet.
"""

CRITIQUE_PROMPT = """
Read `research_plan.md` and evaluate the research plan quality.

Score each dimension (0-25 points):
1. **Comprehensiveness**: Do sub-questions cover all important aspects?
2. **Specificity**: Are questions concrete enough to search effectively?
3. **Diversity**: Do questions explore different angles/perspectives?
4. **Structure**: Is the plan logically organized?

Write your evaluation to `critique.md` with:
- Scores for each dimension
- Specific suggestions for improvement
- Final line MUST be: `SCORE: XX/100`
"""

IMPROVE_PROMPT = """
Read `critique.md` and improve `research_plan.md` based on the feedback.
Address each suggestion while keeping the good parts of the original plan.
"""

def parse_score(filepath):
    """Extract score from critique file."""
    try:
        with open(filepath, "r") as f:
            content = f.read()
        match = re.search(r"SCORE:\s*(\d+)", content, re.IGNORECASE)
        return float(match.group(1)) if match else 0.0
    except FileNotFoundError:
        return 0.0

# ===========================================
# PHASE 1: Planning (with iterative critique)
# ===========================================

research_conversation = Conversation(agent=agent, workspace=cwd)

if resume_from in [None, "planning"]:
    print("=" * 60)
    print("PHASE 1: Creating initial research plan...")
    print("=" * 60)
    
    research_conversation.send_message(PLAN_CREATION_PROMPT.format(topic=topic))
    research_conversation.run()
    save_checkpoint("planning", iteration=0)
    
    # Iterative refinement loop
    start_iteration = checkpoint.get('iteration', 0) if resume_from == "planning" else 0
    for iteration in range(start_iteration, MAX_ITERATIONS):
        print(f"\n{'=' * 60}")
        print(f"ITERATION {iteration + 1}: Critique agent evaluating plan...")
        print("=" * 60)
        
        critique_conversation = Conversation(agent=critique_agent, workspace=cwd)
        critique_conversation.send_message(CRITIQUE_PROMPT)
        critique_conversation.run()
        
        score = parse_score("critique.md")
        print(f"\n→ Plan quality score: {score}/100")
        
        if score >= QUALITY_THRESHOLD:
            print(f"✓ Plan approved! (threshold: {QUALITY_THRESHOLD})")
            break
        else:
            print(f"✗ Below threshold ({QUALITY_THRESHOLD}). Improving...")
            research_conversation.send_message(IMPROVE_PROMPT)
            research_conversation.run()
            save_checkpoint("planning", iteration=iteration+1)
else:
    print("⏭️  Skipping Phase 1 (already complete)")

# ===========================================
# PHASE 2: Research (execute searches)
# ===========================================

if resume_from in [None, "planning", "research"]:
    print(f"\n{'=' * 60}")
    print("PHASE 2: Executing research plan...")
    print("=" * 60)
    
    EXECUTE_PROMPT = """
    Read `research_plan.md` and execute each task:

    1. For each unchecked task (`- [ ]`):
       - Use TavilySearch to find information
       - Add the RAW findings (with source URLs) under each task
       - Mark the task complete (`- [x]`)

    Keep all findings in `research_plan.md` - this is your research notes.
    Do NOT write a summary yet - just gather all the information.
    """
    
    research_conversation.send_message(EXECUTE_PROMPT)
    research_conversation.run()
    save_checkpoint("research")
    
    print("\n✓ Research gathering complete!")
else:
    print("⏭️  Skipping Phase 2 (already complete)")

In [ ]:
# ===========================================
# PHASE 3: Synthesis (comprehensive report)
# ===========================================

print(f"\n{'=' * 60}")
print("PHASE 3: Synthesizing comprehensive report (GPT-5.1)...")
print("=" * 60)

SYNTHESIS_PROMPT = """
You are a research synthesis expert. Read `research_plan.md` which contains:
- The original research question
- Sub-questions that were investigated
- Raw findings from web searches with source URLs

Your task: Write a comprehensive research report to `research_report.md`

## Report Structure:

# [Research Topic Title]

## Executive Summary
(2-3 paragraph overview of key findings and conclusions)

## 1. Introduction
(Context, why this topic matters, scope of research)

## 2. Key Findings

### 2.1 [Theme 1 from the research]
(Synthesize related findings into coherent paragraphs, not bullet points)
(Cite sources inline like: "According to [Source Name](url)...")

### 2.2 [Theme 2]
(Continue for each major theme discovered)

### 2.3 [Theme 3]
...

## 3. Analysis & Implications
(What do these findings mean? Trends, patterns, implications for the future)

## 4. Limitations
(What wasn't covered? What uncertainties remain?)

## 5. Conclusion
(Final synthesis and key takeaways)

## References
(List all sources with URLs)

---

IMPORTANT:
- Write in a professional, analytical tone
- Synthesize and connect ideas - don't just list facts
- Every claim should reference a source
- Be comprehensive but concise
- Make it feel like a real research report, not a summary of search results
"""

# Use the critique agent (GPT-5.1) for synthesis - it has stronger reasoning
synthesis_conversation = Conversation(agent=critique_agent, workspace=cwd)
synthesis_conversation.send_message(SYNTHESIS_PROMPT)
synthesis_conversation.run()

# Clear checkpoint on successful completion
clear_checkpoint()
print("\n✓ Research report complete!")
print("✓ Checkpoint cleared (research finished successfully)")

## Step 7: View the Comprehensive Research Report

GPT-5.1 synthesized all findings into a professional `research_report.md`:

In [ ]:
from IPython.display import Markdown, display

# Show the comprehensive research report (synthesized by GPT-5.1)
try:
    with open("research_report.md", "r") as f:
        display(Markdown(f.read()))
except FileNotFoundError:
    print("research_report.md not found - run the cells above first")

## Architecture Summary

```
┌─────────────────────────────────────────────────────────────┐
│  PHASE 1: PLANNING (with iterative critique)                │
├─────────────────────────────────────────────────────────────┤
│  GPT-4o creates plan → GPT-5.1 critiques → improve → loop   │
│  Output: research_plan.md (approved sub-questions)          │
└─────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────┐
│  PHASE 2: RESEARCH (gather raw findings)                    │
├─────────────────────────────────────────────────────────────┤
│  GPT-4o executes Tavily searches for each sub-question      │
│  Output: research_plan.md (with raw findings + URLs)        │
└─────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────┐
│  PHASE 3: SYNTHESIS (comprehensive report)                  │
├─────────────────────────────────────────────────────────────┤
│  GPT-5.1 reads all findings and writes professional report  │
│  Output: research_report.md (executive summary, analysis,   │
│          themed sections, implications, citations)          │
└─────────────────────────────────────────────────────────────┘
```

### Why Two Models?

| Model | Role | Why |
|-------|------|-----|
| GPT-4o | Research Agent | Fast, good at tool use, searches |
| GPT-5.1 | Critique + Synthesis | Stronger reasoning, better writing |

## OpenHands SDK Features Used

| Feature | How It's Used |
|---------|--------------|
| **Multi-Agent** | GPT-4o (research) + GPT-5.1 (critique & synthesis) |
| **Iterative Refinement** | Plan critiqued and improved until threshold met |
| **File-based State** | `research_plan.md` → `research_report.md` pipeline |
| **Checkpointing** | Resume from crash via `checkpoint.json` |
| **Observability** | Laminar tracing for debugging (set `LMNR_PROJECT_API_KEY`) |

### Output Files

| File | Purpose |
|------|---------|
| `research_plan.md` | Working document with sub-questions + raw findings |
| `critique.md` | Plan evaluation feedback (debugging) |
| `research_report.md` | **Final deliverable** - comprehensive report |
| `checkpoint.json` | Resume state (auto-deleted on success) |